MODEL TESTING

In [25]:
import pandas as pd
import joblib

# ----------------------------------
# 1. LOAD MODELS + PREPROCESSORS
# ----------------------------------
all_training_columns = joblib.load("all_training_columns.pkl")
best_clf_model = joblib.load("best_clf_model.pkl")
best_reg_model = joblib.load("best_reg_model.pkl")
label_encoder = joblib.load("label_encoder.pkl")
scaler_clf = joblib.load("scaler_clf.pkl")
scaler_reg = joblib.load("scaler_reg.pkl")

# ----------------------------------
# 2. LOAD YOUR CLEANED DATASET
# ----------------------------------
df = pd.read_csv("StudentPerformanceFactors_Cleaned.csv")

# ----------------------------------
# 3. APPLY DUMMIES TO MATCH TRAINING FEATURES
# ----------------------------------
df_encoded = pd.get_dummies(df)

# Add missing columns
for col in all_training_columns:
    if col not in df_encoded.columns:
        df_encoded[col] = 0

# Remove extra columns
df_encoded = df_encoded[all_training_columns]

# ----------------------------------
# 4. SCALE NUMERIC FEATURES
# ----------------------------------
df_scaled_clf = scaler_clf.transform(df_encoded)
df_scaled_reg = scaler_reg.transform(df_encoded)

# ----------------------------------
# 5. MAKE PREDICTIONS
# ----------------------------------
clf_pred = best_clf_model.predict(df_scaled_clf)
reg_pred = best_reg_model.predict(df_scaled_reg)

# Convert classification back to labels
clf_pred_label = label_encoder.inverse_transform(clf_pred)

print("Classification Output:", clf_pred_label)
print("Regression Output:", reg_pred)


Classification Output: ['C' 'C' 'B' ... 'C' 'C' 'C']
Regression Output: [67.12037427 60.81325514 74.21946221 ... 67.74728089 68.40001487
 63.93818535]


/usr/local/lib/python3.12/dist-packages/sklearn/preprocessing/_label.py:151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
